雖然預設的即時執行模式（Eager Execution）為我們帶來了靈活及容易偵錯的特性，但在特定的場合，例如追求高性能或部署模型時，我們依然希望使用 TensorFlow 1.X 中預設的圖執行模式（Graph Execution），將模型轉換為高效的 TensorFlow 圖模型。此時，TensorFlow 2 為我們提供了 tf.function 模組，結合 AutoGraph 機制，使得我們僅需加入一個簡單的 @tf.function 修飾符，就能輕鬆將模型以圖執行模式運行。

並不是任何函數都可以被 @tf.function 修飾！@tf.function 使用靜態編譯將函數內的程式碼轉換成計算圖，因此對函數內可使用的語句有一定限制（僅支援 Python 語言的一個子集），且需要函數內的操作本身能夠被建構為計算圖。建議在函數內只使用 TensorFlow 的原生操作，不要使用過於複雜的 Python 語句，函數參數只包括 TensorFlow 張量或 NumPy 陣列，並最好是能夠按照計算圖的思想去建構函數（換言之，@tf.function 只是給了你一種更方便的寫計算圖的方法，而不是一顆能給任何函數加速的 銀子彈 ）。詳細內容可參考 AutoGraph Capabilities and Limitations 。建議配合 附錄 一同閱讀本節以獲得較深入的理解。

In [ ]:
import tensorflow as tf
import time
from zh.model.mnist.cnn import CNN
from zh.model.utils import MNISTLoader

num_batches = 1000
batch_size = 50
learning_rate = 0.001
data_loader = MNISTLoader()
model = CNN()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

@tf.function
def train_one_step(X, y):    
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        # 注意這裡使用了TensorFlow內建的tf.print()。@tf.function不支援Python內建的print方法
        tf.print("loss", loss)
    grads = tape.gradient(loss, model.variables)    
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

start_time = time.time()
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    train_one_step(X, y)
end_time = time.time()
print(end_time - start_time)

**tf.function 內在機制**

當被 @tf.function 修飾的函數第一次被呼叫的時候，進行以下操作：

在即時執行模式關閉的環境下，函數內的程式碼依次運行。也就是說，每個 tf. 方法都只是定義了計算節點，而並沒有進行任何實質的計算。這與 TensorFlow 1.X 的圖執行模式是一致的；

使用 AutoGraph 將函數中的 Python 控制語句轉換成 TensorFlow 計算圖中的對應節點（比如說 while 和 for 語句轉換為 tf.while ， if 語句轉換為 tf.cond 等等；

基於上面的兩步，建立函數內程式碼的計算圖表示（為了保證圖的計算順序，圖中還會自動加入一些 tf.control_dependencies 節點）；

運行一次這個計算圖；

基於函數的名字和輸入的函數參數的類型生成一個雜湊值，並將建立的計算圖緩衝區到一個雜湊表中。

以下是一個測試題：

In [ ]:
import tensorflow as tf
import numpy as np

@tf.function
def f(x):
    print("The function is running in Python")
    tf.print(x)

a = tf.constant(1, dtype=tf.int32)
f(a)
b = tf.constant(2, dtype=tf.int32)
f(b)
b_ = np.array(2, dtype=np.int32)
f(b_)
c = tf.constant(0.1, dtype=tf.float32)
f(c)
d = tf.constant(0.2, dtype=tf.float32)
f(d)

The function is running in Python
1
2
2
The function is running in Python
0.1
0.2


**AutoGraph：將 Python 控制流轉換為 TensorFlow 計算圖**

前面提到，@tf.function 使用名為 AutoGraph 的機制將函數中的 Python 控制流語句轉換成 TensorFlow 計算圖中的對應節點。以下是一個範例，使用 tf.autograph 模組的低層 API tf.autograph.to_code 將函數 square_if_positive 轉換成 TensorFlow 計算圖

In [1]:
import tensorflow as tf

@tf.function
def square_if_positive(x):
    if x > 0:
        x = x * x
    else:
        x = 0
    return x

a = tf.constant(1)
b = tf.constant(-1)
print(square_if_positive(a), square_if_positive(b))
print(tf.autograph.to_code(square_if_positive.python_function))

tf.Tensor(1, shape=(), dtype=int32) tf.Tensor(0, shape=(), dtype=int32)
def tf__square_if_positive(x):
    with ag__.FunctionScope('square_if_positive', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (x,)

        def set_state(vars_):
            nonlocal x
            (x,) = vars_

        def if_body():
            nonlocal x
            x = (ag__.ld(x) * ag__.ld(x))

        def else_body():
            nonlocal x
            x = 0
        ag__.if_stmt((ag__.ld(x) > 0), if_body, else_body, get_state, set_state, ('x',), 1)
        try:
            do_return = True
            retval_ = ag__.ld(x)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)

